# OIE

In [1]:
!pip install nltk

You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install networkx

     |████████████████████████████████| 1.6 MB 20.7 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip install spacy==2.3.0 --use-feature=2020-resolver
import spacy
from spacy.matcher import Matcher

You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [4]:
!spacy download en_core_web_lg
!spacy download fr_core_news_lg

     |████████████████████████████████| 782.7 MB 97.7 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.3.0-py3-none-any.whl size=782931389 sha256=83e8581288a6def8d158b83a262541645ef32505d120372d6ee4054ab55acbeb
  Stored in directory: /tmp/pip-ephem-wheel-cache-dhq21m8k/wheels/ee/b2/51/53c2f8d3c47fb655bf32e3683f15ab41805108d06e76051683
Successfully built en-core-web-lg
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_lg')


In [5]:
import os
import json
import requests
import sys

Each json are put into a dictionary which are put in a list 
So we have a list of dictionnary

Could also be in a pandas frame

The keys are:
* PublishedFirstTimeAt
* SubjectLabels
* Breadcrumb
* FirstPublishedCanonicalWebLink
* BodyParagraphs

In [6]:
FILE_PATH = "devoir/ift6285-le-devoir-articles/"
list_of_files  = os.listdir(FILE_PATH)

list_load_files = []

for file_ in list_of_files:

    json_file_path = FILE_PATH + file_

    with open(json_file_path, 'r') as j:
        contents = json.loads(j.read())

    list_load_files.append(contents)

In [7]:
text_by_articles = []

for i in range(0,len(list_load_files)):
    #for i in range(0,50):
    eleme = list_load_files[i]['BodyParagraphs']
    
    for j, part in enumerate(eleme):
        eleme[j] =  part.replace(u'\xa0', u' ').replace(u'\u2009', u' ')

    if(len(eleme) > 0):
        text_by_articles.append(" ".join(eleme))

# Open information extraction

In [8]:
import fr_core_news_lg

nlp_french = fr_core_news_lg.load()

from nltk import tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
def print_tag(sentence):
    doc = nlp_french(sentence)

    for tok in doc: 
        print(tok.text, "-->",tok.dep_, "-->",tok.pos_)

# With matcher

In [10]:
def get_longest_match(doc, matches):
    max_length = 0
    max_id = 0
    
    for i,tup in enumerate(matches):
        if(max_length < (tup[2] - tup[1])):
            max_length = tup[2] - tup[1]
            max_id = i

    span = doc[matches[max_id][1]:matches[max_id][2]]

    return max_id, span

def extract_information(original_sentence, matcher_nsubj_, matcher_root_):
    
    doc = nlp_french(original_sentence)
    
    matches_n = matcher_nsubj_(doc) 
    matches_r = matcher_root_(doc)

    span = "" 

    if len(matches_n)>0 and len(matches_r)>0:
        index_id_n, span_n = get_longest_match(doc, matches_n)
        index_id_r, span_r = get_longest_match(doc, matches_r)

        if matches_n[index_id_n][2] < matches_r[index_id_r][2]:
            span += str(span_n) + ' ' + str(span_r)

    return span

#=================================================================  

# put a regex for the nsubj cause got a case : nsubj:pass

pattern_BOS_1 = [{'DEP':'det', 'OP':"*"},
            {'DEP':'nsubj', 'OP':"+"},
            {"DEP": {"REGEX": "/nsubj/"}, 'OP':"+"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP':'case', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'nmod', 'OP':"*"},
            {'POS':'NOUN', 'OP':"*"}]


pattern_BOS_2 = [{'DEP':'det', 'OP':"*"},
            {"DEP": {"REGEX": "nsubj"}, 'OP':"+"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP':'case', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'nmod', 'OP':"*"},
            {'POS':'PROPN', 'OP':"*"}]

            
pattern_EOF_1 = [{"LEMMA": 'être'},
            {"DEP": 'det', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'ROOT', 'OP':"+"},
            {'DEP':'case', 'OP':"?"},
            {'DEP':'det', 'OP':"?"},
            {'DEP':'case', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'nmod', 'OP':"*"}]


pattern_EOF_2 = [{"LEMMA": 'être'},
            {"DEP": 'det', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'ROOT', 'OP':"+"},
            {'DEP':'case', 'OP':"?"},
            {'DEP':'det', 'OP':"?"},
            {'DEP':'case', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'nmod', 'OP':"*"},
            {'POS': 'NOUN', 'OP':"*"},
            {'DEP': 'amod', 'OP':"*"}]


matcher_nsubj1 = Matcher(nlp_french.vocab) 
#matcher_nsubj.add("matching_2", None, pattern_2)
matcher_nsubj1.add("matching_1", None, pattern_BOS_1)
matcher_nsubj1.add("matching_2", None, pattern_BOS_2)

matcher_root1 = Matcher(nlp_french.vocab) 
#matcher_root1.add("matching_3", None, pattern_EOF_1)
matcher_root1.add("matching_4", None, pattern_EOF_2)

In [11]:
pattern_BOS_1 = [{'DEP':'det', 'OP':"*"},
            {'DEP': 'amod', 'OP':"*"},
            {"DEP": {"REGEX": "nsubj"}, 'OP':"+"},
            {'POS':'NOUN', 'OP':"*"},
            {'DEP': 'amod', 'OP':"*"}]


pattern_BOS_2 = [{'DEP':'det', 'OP':"*"},
            {'DEP': 'amod', 'OP':"*"},
            {"DEP": {"REGEX": "nsubj"}, 'OP':"+"},
            {'POS':'PROPN', 'OP':"*"},
            {'DEP': 'amod', 'OP':"*"}]

            
pattern_EOF_1 = [{"LEMMA": 'être'},
            {"DEP": 'det', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'DEP': 'ROOT', 'OP':"+"},
            {'DEP':'case', 'OP':"?"},
            {'DEP':'det', 'OP':"?"},
            {'DEP': 'amod', 'OP':"*"},
            {'POS': 'NUM', 'OP':"*"},
            {'POS': 'PROPN', 'OP':"*"},
            {'POS': 'NOUN', 'OP':"*"},
            {'POS': 'NUM', 'OP':"*"}]


matcher_nsubj = Matcher(nlp_french.vocab) 
#matcher_nsubj.add("matching_2", None, pattern_2)
matcher_nsubj.add("matching_1", None, pattern_BOS_1)
matcher_nsubj.add("matching_2", None, pattern_BOS_2)

matcher_root = Matcher(nlp_french.vocab) 
matcher_root.add("matching_3", None, pattern_EOF_1)

# With tree dependency

In [12]:
def get_neighbors_noun(element):

    extract = []


    for child_root in element.lefts: 
        if child_root.dep_ in ['amod','det', 'case', 'nummod'] or child_root.pos_ in ['NOUN','PROPN']:
            extract.append(child_root.text)
    extract.append(element.text)


    for child_root in element.rights: 
        if child_root.dep_ in ['amod','det', 'case', 'nummod'] or child_root.pos_ in ['NOUN','PROPN']:
            extract.append(child_root.text) 

    return extract

def get_neighbors_ROOT(root):

    suffix = []
    prefix = []

    for child_root in root.lefts:
        if child_root.dep_ in ['amod']:
            suffix.append(child_root.text)

    suffix.append(root.text)

    for child_root in root.rights:
        if child_root.dep_ in ['amod']:
            suffix.append(child_root.text)

        if (child_root.dep_ in ['nmod']) and child_root.pos_ in ['NOUN','PROPN', 'NUM']:

            for elem in get_neighbors_noun(child_root):
                suffix.append(elem)

    for child_root in root.lefts:

        if (child_root.dep_ in ['nsubj'] or child_root.dep_ in ['nsubj:pass']) and child_root.pos_ in ['NOUN','PROPN', 'NUM']:
            for elem in get_neighbors_noun(child_root):
                prefix.append(elem)

    return suffix, prefix


def extraction_with_adj(new_sentence):

    entity1 = None
    entity2 = None

    doc = nlp_french(new_sentence)
    sent = doc.sents

    prefix = []
    suffix = []

    window = 4

    for w in sent:

        count = 0

        for child in w.root.children:

            if child.lemma_ == 'être':

                subject, obj = '', ''

                suffix, prefix = get_neighbors_ROOT(w.root)

                middle = child.text
                for i in range(1, window+1):
                    if((count+i)<len(w)):
                        if(str(w[count+i]) == "pas"):
                            middle = child.text + " pas"
                break

            count += 1


        if (w.root.pos_ == 'NOUN' or w.root.pos_ == 'PROPN' or w.root.pos_ == 'ADJ' or w.root.pos_ == 'VERB') and len(prefix) > 0 and len(suffix) > 0: #and entity2 is not None:
            extract = ' '.join(prefix) + ' ' + middle + ' ' + ' '.join(suffix)

        else:
            return []

    return extract


def extraction2(new_sentence):

    entity1 = None
    entity2 = None

    doc = nlp_french(new_sentence)
    sent = doc.sents

    prefix = []
    suffix = []

    window = 4

    for w in sent:

        count = 0

        for child in w.root.children:

            if child.lemma_ == 'être':

                subject, obj = '', ''

                suffix, prefix = get_neighbors_ROOT(w.root)

                middle = child.text
                for i in range(1, window+1):
                    if((count+i)<len(w)):
                        if(str(w[count+i]) == "pas"):
                            middle = child.text + " pas"
                break

            count += 1


        if (w.root.pos_ == 'NOUN' or w.root.pos_ == 'PROPN') and len(prefix) > 0 and len(suffix) > 0: #and entity2 is not None:
            extract = ' '.join(prefix) + ' ' + middle + ' ' + ' '.join(suffix)

        else:
            return []

    return extract

In [13]:
#from spacy import displacy 
#displacy.render(nlp_french(str("Fils d’un historien ultrasioniste, M. Nétanyahou est né en 1949, juste après la création de l’État d’Israël.")), style='dep', jupyter = True, options = {'distance': 120})

In [14]:
oie_with_tree = []

count_ = 0

for i in range(40):
    for sentence in tokenize.sent_tokenize(text_by_articles[60+i]):
        extract = extraction_with_adj(sentence)
        if len(extract) > 0:
            print(sentence)
            print(extract)
            print("\n")
            count_ += 1
            oie_with_tree.append(extract)

print(count_)

Mme Nauert n’avait jamais été officiellement nommée à ce poste.
Mme Nauert été nommée


Mme Nauert est une ancienne journaliste de la chaîne Fox News.
Mme Nauert est ancienne journaliste de la chaîne Fox


« L’état dans lequel se trouve le gouvernement est inacceptable », a estimé dimanche soir la présidente du parti SPD, Andrea Nahles, en référence aux querelles continuelles qui opposent la chancelière à son aile droite sur la politique migratoire.
L’ état est inacceptable


Les événements ont été déplacés à la plaine des Jeux en raison des travaux sur l’amphithéâtre naturel, rappelle-t-il : « On est là pour une période transitoire en attendant un autre site.
Les événements été déplacés


Plusieurs motifs sont avancés pour justifier cette mystification : la NASA serait incapable d’un tel exploit technologique ; il s’agissait de missions sans équipage humain ; d’autres enfin, affirment qu’il fallait dissimuler la complicité d’extraterrestres ou la découverte d’une civilisation lunaire…

In [15]:
len(oie_with_tree)

136

In [16]:
oie_with_tree = []

count_ = 0

for i in range(40):
    print(i)
    for sentence in tokenize.sent_tokenize(text_by_articles[60+i]):
        extract = extraction_with_adj(sentence)
        if len(extract) > 0:
            print(sentence)
            print(extract)
            print("\n")
            count_ += 1
            oie_with_tree.append(extract)

print(count_)

0
Mme Nauert n’avait jamais été officiellement nommée à ce poste.
Mme Nauert été nommée


Mme Nauert est une ancienne journaliste de la chaîne Fox News.
Mme Nauert est ancienne journaliste de la chaîne Fox


1
« L’état dans lequel se trouve le gouvernement est inacceptable », a estimé dimanche soir la présidente du parti SPD, Andrea Nahles, en référence aux querelles continuelles qui opposent la chancelière à son aile droite sur la politique migratoire.
L’ état est inacceptable


2
Les événements ont été déplacés à la plaine des Jeux en raison des travaux sur l’amphithéâtre naturel, rappelle-t-il : « On est là pour une période transitoire en attendant un autre site.
Les événements été déplacés


3
4
Plusieurs motifs sont avancés pour justifier cette mystification : la NASA serait incapable d’un tel exploit technologique ; il s’agissait de missions sans équipage humain ; d’autres enfin, affirment qu’il fallait dissimuler la complicité d’extraterrestres ou la découverte d’une civilisatio

In [17]:
len(oie_with_tree)

136

In [18]:
oie = []
for i in range(60):
    for sentence in tokenize.sent_tokenize(text_by_articles[i]):
        extract = extract_information(sentence, matcher_nsubj, matcher_root)
        if len(extract) > 0:
            print(sentence)
            print(extract)
            print("\n")
            oie.append(extract)

Il a été intronisé au Temple de la renommée du football canadien en 2012.
Il été intronisé au Temple


Wetenhall est propriétaire des Alouettes depuis plus de 20 ans.
Wetenhall est propriétaire des Alouettes


La vente de produits de vapotage a été interdite aux Canadiens de moins de 18 ans et toute promotion visant à rendre ces produits attrayants pour les jeunes ou à faire la promotion d’un style de vie a été bannie.
La vente été interdite aux Canadiens


Depuis 2015, les produits de vapotage sont considérés comme des produits du tabac dans la province, ce qui restreint énormément toute activité promotionnelle.
les produits sont considérés comme des produits


Cette proportion était de 9 % en 2014-2015 ; une hausse de 64 % a donc été observée en deux ans.
Cette proportion été observée en deux ans


L’annonce d’Ottawa a été saluée par des organismes canadiens de lutte antitabac.
L’annonce été saluée par des organismes


Du côté des progressistes, on s'est réjoui tout en prenant soin d

In [19]:

count_ = 0

for i in range(len(text_by_articles)):
    for sentence in tokenize.sent_tokenize(text_by_articles[i]):
        count_ += 1

print(count_)

276425


# Ensemble test

In [24]:
FILE_PATH = "test-projet2/"
list_of_files  = os.listdir(FILE_PATH)

list_load_files = []

for file_ in list_of_files:

    json_file_path = FILE_PATH + file_

    with open(json_file_path, 'r') as j:
        contents = json.loads(j.read())

    list_load_files.append(contents)

In [25]:
text_by_articles = []

for i in range(0,len(list_load_files)):
    #for i in range(0,50):
    eleme = list_load_files[i]['BodyParagraphs']
    
    for j, part in enumerate(eleme):
        eleme[j] =  part.replace(u'\xa0', u' ').replace(u'\u2009', u' ')

    if(len(eleme) > 0):
        text_by_articles.append(" ".join(eleme))

print(len(text_by_articles), "articles found")

5 articles found


In [ ]:
count_ = 0

for i in range(len(text_by_articles)):
    for sentence in tokenize.sent_tokenize(text_by_articles[i]):
        count_ += 1

print(count_)

In [37]:
oie_test = []

for i in range(len(text_by_articles)):
    for sentence in tokenize.sent_tokenize(text_by_articles[i]):
        extract = extraction_with_adj(sentence)
        if len(extract) > 0:
            print(sentence)
            print(extract)
            print("\n")
            oie_test.append(extract)

La compétition est forte entre un après-midi au parc et la visite d’une des galeries du Musée des beaux-arts de Montréal (MBAM), mais Pascale Chassé, directrice des communications au MBAM, ne s’en formalise pas trop : « Le beau temps nous amène quand même des visiteurs, surtout lorsqu’il y a des canicules et que les gens cherchent de l’air conditionné », lance-t-elle en riant.
La compétition est forte


Ce Jardin de sculptures, oasis de génie créatif, est un endroit pour prendre une pause de la ville, suggère Pascale Chassé.
Ce Jardin sculptures est endroit


Étonné de la réponse, le juge Nollet s’est alors interrogé sur la crédibilité du document.
le juge Nollet est interrogé


Les Mustangs sont deuxièmes avec sept.
Les Mustangs sont deuxièmes


Le match de la Coupe Vanier sera présenté le 24 novembre à Québec.
Le match Coupe sera présenté


« Ce meurtre brutal d'un journaliste, d'un homme innocent, d'un dissident, ne sera pas laissé sans une réponse américaine, et je l'espère, sans u